In [1]:
import os
import sys

import tqdm

In [2]:
FILEPATH_IN = "../data/gisaid_hcov-19_2020_11_29_high_qualily.fasta"
FILEPATH_OUT = "../data/filtered_gisaid_hcov-19_2020_11_29_high_qualily.fasta"
FILEPATH_EXCLUDE = '../data/exclude.txt'
EXCLUDING_ANIMALS = [
    'lion',
    'tiger',
    'mink',
    'env',
    'cat',
    'pangolin',
    'canine',
    'mouse',
]
MAX_UNKNOWN_FRACTION = .05
MIN_SEQ_LENGHT = 29700

In [5]:
EXCLUDE_IDX = set()
with open(FILEPATH_EXCLUDE) as fin:
    for line in fin:
        if not line.startswith('#') and line != '\n':
            EXCLUDE_IDX.add(line.strip())

In [8]:
def one_seq_writer(header, seq_lst, fout):
    """return 1 if seq was not written, else 0"""
    name, idx, date = header.split('|')
    seq = ''.join(map(str.strip, seq_lst))
    n_fraction = (seq.count('N') + seq.count('n')) / len(seq)
    drop_access = any([
        n_fraction > MAX_UNKNOWN_FRACTION,
        name.split('/')[1] in EXCLUDING_ANIMALS,
        name.lstrip(">hCoV-19/") in EXCLUDE_IDX,
        len(seq) < MIN_SEQ_LENGHT,
    ])
    if drop_access:
        return 1
    
    fout.write(header)
    fout.write(''.join(seq_lst))
    return 0    

In [9]:
n_seq_dropped = 0

with open(FILEPATH_OUT, 'a') as fout:
    with open(FILEPATH_IN) as fasta_fin:
        for i, line in tqdm.tqdm(enumerate(fasta_fin)):
            if line.startswith('>'):
                if i > 0:
                    res = one_seq_writer(header, seq_lst, fout)
                    n_seq_dropped += res

                header = line
                seq_lst = []
            else:
                seq_lst.append(line)
    # last seq
    res = one_seq_writer(header, seq_lst, fout)
    n_seq_dropped += res

62957145it [01:38, 640861.14it/s]


In [10]:
n_seq_dropped

4574